# Dockerfile Breakdown and Documentation

Generated on 2024-10-29

This notebook provides a detailed explanation of each section of the Dockerfile, including commands, conditions, and chained logic to help understand each line of code.

## **Section 1: Base Image & Environment Setup**

This section sets up the base image, environment variables, and working directory.

Each line is configured to ensure efficient operation, reduced image size, and effective logging during container execution.

In [ ]:
```
# Use Python 3.11 slim as the base image to keep the container lightweight.
# Slim variants remove unnecessary tools, reducing image size.
FROM python:3.11-slim

# Set the environment variable to avoid interactive prompts during installations.
ENV DEBIAN_FRONTEND=noninteractive

# Disable Python’s output buffering to ensure that logs are displayed instantly.
ENV PYTHONUNBUFFERED=1

# Prevent Python from generating .pyc files to save space.
ENV PYTHONDONTWRITEBYTECODE=1

# Set the working directory to /app.
WORKDIR /app
```


## **Section 2: System Packages Installation**

In this section, necessary system packages are installed using `apt-get`. These packages support essential operations within the container.

Commands are chained with `&&` to ensure each command runs successfully before moving to the next, and the apt cache is cleared to reduce image size.

In [ ]:
```
# Update the package list and install essential packages for building and running software.
RUN apt-get update && apt-get install -y \
    wget \
    bzip2 \
    ca-certificates \
    build-essential \
    cmake \
    && rm -rf /var/lib/apt/lists/*
```


## **Section 3: Mambaforge Installation (Miniconda Alternative)**

This block downloads and installs the Mambaforge installer based on the architecture of the system (either x86_64 or ARM). Using a conditional `if-else` structure, the script fetches the appropriate installer and performs cleanup after installation.

In [ ]:
```
# Detect system architecture and install appropriate Miniforge version.
RUN arch=$(uname -m) && \
    if [ "${arch}" = "x86_64" ]; then \
        wget -q "https://github.com/conda-forge/miniforge/releases/latest/download/Miniforge3-Linux-x86_64.sh" -O miniforge.sh; \
    elif [ "${arch}" = "aarch64" ]; then \
        wget -q "https://github.com/conda-forge/miniforge/releases/latest/download/Miniforge3-Linux-aarch64.sh" -O miniforge.sh; \
    else \
        echo "Unsupported architecture: ${arch}"; \
        exit 1; \
    fi && \
    bash miniforge.sh -b -p /opt/miniforge && \
    rm miniforge.sh
```


## **Section 4: Environment Setup with Mamba**

The Mambaforge installation adds `mamba` to the PATH, and an environment named 'team3_env' is created with Python 3.11. The default shell is changed to run commands within this new environment.

In [ ]:
```
# Add Mambaforge to PATH and create new environment.
ENV PATH=/opt/miniforge/bin:$PATH
RUN mamba create -n team3_env python=3.11 -y
SHELL ["mamba", "run", "-n", "team3_env", "/bin/bash", "-c"]
```


## **Section 5: Install Project Dependencies**

The project's dependencies are specified in `requirements.txt` and additional packages are installed using `pip` for libraries not included in the requirements file. This block also cleans up package caches after installation to save space.

In [ ]:
```
# Copy requirements.txt and install dependencies.
COPY requirements.txt /app/requirements.txt
RUN mamba install --yes --file requirements.txt && mamba clean --all -f -y
RUN pip install rank_bm25 streamlit-pdf-viewer
```


## **Section 6: Copy Application Files and Expose Ports**

The entire app directory is copied to `/app` inside the container, and necessary ports are exposed to allow external access for Streamlit and Jupyter Notebook.

In [ ]:
```
# Copy files and expose necessary ports.
COPY . /app
EXPOSE 5003
EXPOSE 6003
```


## **Section 7: Update PATH for Conda Environment**

By adding the new environment's binary path to the PATH variable, any command executed will use the dependencies and tools installed in 'team3_env' by default.

In [ ]:
```
# Add team3_env environment’s bin directory to PATH.
ENV PATH=/opt/miniforge/envs/team3_env/bin:$PATH
```


## **Section 8: Define Entry Point and Default Command**

Sets the entry point to Python, allowing scripts to run with `python` by default. The `CMD` instruction tells the container to run `app.py` if no other command is specified at runtime.

In [ ]:
```
# Set default entry point and command.
ENTRYPOINT ["python"]
CMD ["app.py"]
```
